In [69]:
!pip install -r requirements.txt

In [2]:
# Load data
import pandas as pd 

file_path='assets/datasets/clean/250604-consolidated_survey.xlsx'
df_load=pd.read_excel(file_path).drop(columns=['Unnamed: 0'])

In [3]:
df_load.head()

,start_date,end_date,survey_status,completion_level,completion_time,survey_completed,recorded_date,response_id,distribution_channel,user_language,...,soc_aspects,soc_aspects_other,es_assess,es_assess_other,es_more_info,es_certif,email,survey,inv_platform,inv_platform_other
0,2024-11-13 10:30:58,2024-11-13 10:37:24,0,100,386,1,2024-11-13 10:37:25.745,R_2CYvoda8TZhYFaN,anonymous,IT,...,"2,3,4",NaN,2,NaN,3.0,"2,3",NaN,BA,NaN,NaN
1,2024-11-14 14:00:16,2024-11-14 14:08:08,0,100,472,1,2024-11-14 14:08:09.160,R_2PCMoZEjgiOeupr,anonymous,IT,...,1,NaN,2,NaN,3.0,1,NaN,BA,NaN,NaN
2,2024-11-14 14:28:17,2024-11-14 14:36:18,0,100,480,1,2024-11-14 14:36:18.958,R_8HBDPZSIjz1M0rn,anonymous,IT,...,"1,3",NaN,3,NaN,5.0,"2,3",anmata@libero.it,BA,NaN,NaN
3,2024-11-14 16:46:00,2024-11-14 18:31:25,0,100,6324,1,2024-11-14 18:31:26.274,R_8dSEqbK9X2kHC6i,anonymous,IT,...,"1,4",NaN,2,NaN,3.0,1,NaN,BA,NaN,NaN
4,2024-11-14 18:26:48,2024-11-14 18:42:58,0,100,969,1,2024-11-14 18:42:58.484,R_2kq5hZdQfhR6sbI,anonymous,IT,...,"2,3",NaN,2,NaN,4.0,1,francoagostinoperone@gmail.com,BA,NaN,NaN


In [4]:
df_load.columns

Index(['start_date', 'end_date', 'survey_status', 'completion_level',
       'completion_time', 'survey_completed', 'recorded_date', 'response_id',
       'distribution_channel', 'user_language', 'recaptcha_score',
       'gender_male', 'age', 'children_num', 'edu_level', 'edu_field',
       'edu_field_other', 'working_status', 'working_status_other',
       'expertise_area', 'expertise_area_other', 'work_industries',
       'work_industries_other', 'work_company_size', 'roles_cat', 'inv_time',
       'inv_motivation', 'inv_motivation_other', 'ba_group', 'ba_group_name',
       'ba_group_name_other', 'ba_group_reason', 'ba_group_reason_other',
       'inv_num', 'inv_companies_last2y', 'inv_amount_average',
       'inv_capital_stake', 'inv_dd_duration', 'inv_criteria',
       'inv_criteria_other', 'inv_sectors', 'inv_sectors_other', 'inv_process',
       'inv_process_other', 'inv_visits', 'non_fin_contr',
       'non_fin_contr_other', 'child_bias', 'inv_goal', 'inv_exit',
       'inv_re

In [ ]:
# Proportions of women and men in both surveys

pd.options.display.max_columns = None
columns={
    'gender_male': 'gender',
    'start_date': 'percentage',
}
# df_temp=df_load[['survey', 'gender_male', 'age', 'children_num', 'edu_level']].groupby(['survey']).mean()
df_temp=df_load.copy()
df_temp['gender_male']=df_temp['gender_male'].replace(1, 'women')
df_temp['gender_male']=df_temp['gender_male'].replace(2, 'men')
df_temp=df_temp.dropna(subset=['gender_male'])
df_stats=(df_temp.groupby(['survey', 'gender_male'])['start_date'].count()).div(df_temp.groupby(['survey'])['start_date'].count())
df_stats=df_stats.reset_index().rename(columns=columns)
df_stats

,survey,gender,percentage
0,BA,men,0.712871
1,BA,women,0.287129
2,CI,men,0.617021
3,CI,women,0.382979


In [ ]:
# Returns the statistics in funcs on groups and variables columns
import pandas as pd 

def basic_stats(funcs, groups, columns):
    df_results=pd.DataFrame()
    for column in columns:
        df_temp=df_load.dropna(subset=[column]) # Keep only non nan
        df_results=pd.concat([df_results,df_temp.groupby(groups)[column].agg(funcs).reset_index().assign(var=column)])

    return df_results

In [20]:
# Investor experience Numerical and ordinal variables
# Gender

funcs=['mean', 'std', 'count']
groups=['gender_male']
columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']

basic_stats(funcs, groups, columns)

,gender_male,mean,std,count,var
0,1.0,3.064516,1.093480,31,inv_time
1,2.0,3.372093,1.337658,86,inv_time
0,1.0,3.375000,1.539795,32,inv_num
1,2.0,3.423529,1.483051,85,inv_num
0,1.0,1.814815,1.177907,27,inv_amount_average
1,2.0,2.500000,1.295408,74,inv_amount_average
0,1.0,2.230769,1.394495,26,women_led_inv
1,2.0,1.835616,1.027588,73,women_led_inv


In [21]:
# Investor experience Numerical and ordinal variables
# survey

funcs=['mean', 'std', 'count']
groups=['survey']
columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']

basic_stats(funcs, groups, columns)

,survey,mean,std,count,var
0,BA,3.563830,1.187375,94,inv_time
1,CI,2.173913,1.029217,23,inv_time
0,BA,3.760870,1.329046,92,inv_num
1,CI,2.153846,1.347362,26,inv_num
0,BA,2.306818,1.271780,88,inv_amount_average
1,CI,2.384615,1.502135,13,inv_amount_average
0,BA,1.965517,1.175805,87,women_led_inv
1,CI,1.750000,0.866025,12,women_led_inv


In [65]:
from scipy.stats import mannwhitneyu
import pandas as pd 
import numpy as np
from scipy.stats import norm

def mw_test(columns, condition_label, condition_values):
    result_columns=['var', 'u_stat', 'pvalue']
    df_results=pd.DataFrame(columns=result_columns)
    for column in columns:
        
        df_temp=df_load.dropna(subset=[column, condition_label]) # Keep only non nan

        groups=[]
        for i in range(len(condition_values)):
            groups.append(df_temp[df_temp[condition_label]==condition_values[i]].loc[:,column].array)

        u, p=mannwhitneyu(groups[0], groups[1], alternative='two-sided')
        df_results=pd.concat([df_results, pd.DataFrame([[column, u, p]], columns=result_columns)])
    
    return df_results


In [ ]:
# Mann Whitney test on gender_male

columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']
condition_label='gender_male'
condition_values=[1,2]

mw_test(columns, condition_label, condition_values)

C:\Users\Dell\AppData\Local\Temp\ipykernel_19252\4196572552.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results=pd.concat([df_results, pd.DataFrame([[column, u, p]], columns=result_columns)])


,var,u_stat,pvalue
0,inv_time,1155.5,0.261672
0,inv_num,1352.5,0.965182
0,inv_amount_average,682.5,0.011706
0,women_led_inv,1083.0,0.255738


In [67]:
# Mann Whitney test on survey

columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']
condition_label='survey'
condition_values=['BA', 'CI']

mw_test(columns, condition_label, condition_values)

C:\Users\Dell\AppData\Local\Temp\ipykernel_19252\4196572552.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results=pd.concat([df_results, pd.DataFrame([[column, u, p]], columns=result_columns)])


,var,u_stat,pvalue
0,inv_time,1729.5,5.037791e-06
0,inv_num,1948.0,6.397449e-07
0,inv_amount_average,565.5,9.495629e-01
0,women_led_inv,555.0,7.091040e-01


In [ ]:
import plotly.graph_objects as go

funcs=['mean', 'std', 'count']
groups=['gender_male']
columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']

df_stats=basic_stats(funcs, groups, columns)

categories=['Investment Experience', 'Number of Investments', 'Average Invested Amount', 'Number of Women-Led Investments']
r1=df_stats[df_stats[groups[0]]==1]['mean']
r2=df_stats[df_stats[groups[0]]==2]['mean']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=r1,
      theta=categories,
      fill='toself',
      name='Women'
))
fig.add_trace(go.Scatterpolar(
      r=r2,
      theta=categories,
      fill='toself',
      name='Men'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 6]
    )),
  showlegend=False
)

fig.show()

In [78]:
import plotly.graph_objects as go

funcs=['mean', 'std', 'count']
groups=['survey']
columns=['inv_time', 'inv_num', 'inv_amount_average', 'women_led_inv']

df_stats=basic_stats(funcs, groups, columns)

categories=['Investment Experience', 'Number of Investments', 'Average Invested Amount', 'Number of Women-Led Investments']
r1=df_stats[df_stats[groups[0]]=='BA']['mean']
r2=df_stats[df_stats[groups[0]]=='CI']['mean']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=r1,
      theta=categories,
      fill='toself',
      name='Business Angel'
))
fig.add_trace(go.Scatterpolar(
      r=r2,
      theta=categories,
      fill='toself',
      name='Crowdfunding'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 6]
    )),
  showlegend=False
)

fig.show()